In [ ]:
import setup

setup.init()

In [ ]:
import helpers.bright_data

In [ ]:
from snapshots.tasks import perform_reddit_scrape_task

In [ ]:
from reddit.models import RedditCommunity

import ai

In [ ]:
# ai.extract_topics_agent(
#     "Why are there cameras everywhere? Wouldn't the world be better without cameras?"
# )

In [ ]:
query = "Automation and n8n and make.com and zapier.com"

topics = ai.extract_topics_agent(query)
topic_names = ", ".join([x.get("name") for x in topics])

topic_names

In [ ]:
for topic in topics:
    topic_name = topic.get("name")

    print(f"Researching {topic_name}")

    community_data = ai.perform_get_reddit_communities(topic_name)

    for community in community_data:
        url = community.pop("url")

        print(community)

        instance, created = RedditCommunity.objects.update_or_create(
            url=url, defaults=community
        )

        if created:
            for sort_by in helpers.bright_data.BRIGHT_DATA_SCRAPE_SORT_OPTIONS:
                perform_reddit_scrape_task.delay(
                    url=url,
                    num_of_posts=5,
                    progress_countdown=300,
                    sort_by_time=sort_by,
                )
        else:
            perform_reddit_scrape_task.delay(
                url=url,
                num_of_posts=5,
                progress_countdown=300,
                sort_by_time="This Week",
            )